In [1]:
import numpy as np 
import pandas as pd 
import sys
import os
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pylab as plt
import warnings
import itertools
from scipy import stats
warnings.filterwarnings('ignore')

# Add the grandparent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from Utils import FE_helper as FE


c:\Users\yangs\.conda\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. Load the data
train_df_original = pd.read_csv('../../Original_Data/train_2025.csv') 
train_df = pd.read_csv('../../Original_Data/train_2025.csv') 
test_df = pd.read_csv('../../Original_Data/test_2025.csv')

train_df = FE.add_features(train_df)
test_df = FE.add_features(test_df)

test_id = test_df['claim_number']
train_id = train_df['claim_number']
target = train_df['fraud']

ignore_var = ['claim_date.is_weekend', 'claim_date.near_holiday', 'fraud']
train_df = FE.drop_ignored_columns(train_df, ignore_var)
test_df = FE.drop_ignored_columns(test_df, ignore_var)

In [163]:
def filter_low_cardinality_columns(df, threshold=20, dropna=False):
    """
    Filters columns with unique value counts ≤ threshold.
    """
    low_card_cols = [col for col in df.columns if df[col].nunique(dropna=dropna) <= threshold]
    # print(f"Low-cardinality columns (≤ {threshold} unique values): {low_card_cols}")
    return low_card_cols

def generate_column_combinations(columns, sizes=[2, 3]):
    """
    Generates all combinations of the given columns at specified sizes.
    """
    combos = []
    for k in sizes:
        combos.extend(itertools.combinations(columns, k))
    print(f"Generated {len(combos)} combinations (sizes {sizes}).")
    return combos


def add_presence_features(df, combos):
    """
    For each column combination, adds a binary feature:
    1 if the row's combination appears elsewhere in the dataset, 0 otherwise.
    Shows a progress indicator.
    """
    df_out = df.copy()
    total = len(combos)

    for i, combo in enumerate(combos, 1):
        combo_name = "__".join(combo) + "_present"
        
        # Progress indicator
        progress_msg = f"\rProcessing {i} / {total} combos ({100 * i / total:.2f}%)"
        sys.stdout.write(progress_msg)
        sys.stdout.flush()
        
        # Create a tuple column for matching
        combo_tuples = df[list(combo)].apply(tuple, axis=1)
        # Count how many times each tuple appears
        counts = combo_tuples.map(combo_tuples.value_counts())
        # Presence = appears more than once
        df_out[combo_name] = (counts > 1).astype(int)
    
    # Final newline to clean up progress line
    sys.stdout.write("\nDone!\n")

    return df_out

def compute_presence_counts(df, combos):
    """
    Computes the count matrix for each combo (how many times each combo appears).

    Parameters:
    ----------
    df : pd.DataFrame
        Input DataFrame.
    combos : list of tuples
        List of column combinations to create presence features from.

    Returns:
    -------
    pd.DataFrame
        A DataFrame where each column is the counts for that combo.
    """
    df_out = df.copy()
    total = len(combos)

    for i, combo in enumerate(combos, 1):
        combo_name = "__".join(combo) + "_count"

        # Progress indicator
        progress_msg = f"\rProcessing {i} / {total} combos ({100 * i / total:.2f}%)"
        sys.stdout.write(progress_msg)
        sys.stdout.flush()

        combo_tuples = df[list(combo)].apply(tuple, axis=1)
        counts = combo_tuples.map(combo_tuples.value_counts())

        df_out[combo_name] = counts

    sys.stdout.write("\n✅ Done computing counts matrix.\n")
    return df_out

def generate_all_nonempty_subsets(features):
    """
    Generates all nonempty subsets (combinations) of the given feature list.
    
    Parameters:
    - features (list): List of feature names.
    
    Returns:
    - list of tuples: All nonempty subsets.
    """
    all_subsets = []
    for k in range(1, len(features) + 1):
        combos = list(itertools.combinations(features, k))
        all_subsets.extend(combos)
    print(f"Generated {len(all_subsets)} total nonempty subsets.")
    return all_subsets

def compute_fraud_rate_differences(df, target_col='fraud', suffix='_present', variance_threshold=None):
    """
    Computes the difference in fraud rates between rows with feature == 1 and feature == 0
    for all columns ending with the given suffix.
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame
    - target_col (str): Name of the target binary column (e.g., 'fraud')
    - suffix (str): Suffix to identify newly added features
    - filter_zero_variance (bool): Whether to exclude columns with zero variance
    
    Returns:
    - pd.DataFrame: Feature name, fraud rate at 0, fraud rate at 1, and the difference
    """
    results = []

    # Select columns with the specified suffix
    feature_cols = [col for col in df.columns if col.endswith(suffix)]

    if variance_threshold is not None:
        variances = df[feature_cols].var()
        feature_cols = [col for col in feature_cols if variances[col] > variance_threshold]
        print(f"Kept {len(feature_cols)} higher-variance features.")

    for col in feature_cols:
        grouped = df.groupby(col)[target_col].mean()
        rate_0 = grouped.get(0, None)
        rate_1 = grouped.get(1, None)
        if rate_0 is not None and rate_1 is not None:
            diff = rate_1 - rate_0
            var = df[col].var()
            results.append({
                'feature': col,
                'fraud_rate_at_0': rate_0,
                'fraud_rate_at_1': rate_1,
                'difference': diff,
                'variance': var
            })

    results_df = pd.DataFrame(results).sort_values(by='difference', key=abs, ascending=False)
    return results_df

compute_presence_counts takes a ton of resources. here is a quick test case if want to be assured that it works

In [174]:
# df = pd.DataFrame({
#     'gender': ['M', 'F', 'M', 'M', 'F', 'M'],
#     'age':    [25, 30, 25, 40, 30, 25],
#     'city':   ['NY', 'SF', 'NY', 'LA', 'SF', 'NY']
# })

# combos = [
#     ('gender', 'age'),
#     ('gender', 'city')
# ]

# counts_df = compute_presence_counts(df, combos)
# counts_df

In [176]:
additional_drops = ['vehicle_price_categories', 'log_pop_bin', 'age_group', 'past_num_of_claims', 'age_of_vehicle', 'claim_date.weekofyear', 'claim_date.day', 'claim_date.quarter', 'zipcode_type']

low_performing_col = train_df[filter_low_cardinality_columns(train_df, threshold=60)].drop(columns = additional_drops).columns.tolist()

# Step 2: Generate all 2- and 3-column combinations
combos = generate_all_nonempty_subsets(low_performing_col)

# Step 3: Add binary presence features
df_with_features = compute_presence_counts(train_df, combos)

print("New dataset shape:", df_with_features.shape)

Generated 32767 total nonempty subsets.
Processing 32767 / 32767 combos (100.00%)
✅ Done computing counts matrix.
New dataset shape: (18000, 32809)


In [177]:
df_with_features

,claim_number,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,accident_site,...,gender__marital_status__high_education_ind__address_change_ind__living_status__accident_site__witness_present_ind__channel__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__address_change_ind__living_status__accident_site__witness_present_ind__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__address_change_ind__living_status__accident_site__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__address_change_ind__living_status__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__address_change_ind__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__living_status__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__address_change_ind__living_status__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__high_education_ind__address_change_ind__living_status__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,marital_status__high_education_ind__address_change_ind__living_status__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count,gender__marital_status__high_education_ind__address_change_ind__living_status__accident_site__witness_present_ind__channel__policy_report_filed_ind__vehicle_category__vehicle_color__claim_date.year__claim_date.month__claim_date.dayofweek__state_count
0,2759,58,M,1,83,117115,1,0,Own,Local,...,1,1,1,1,1,1,1,1,1,1
1,11338,37,F,0,76,57945,1,0,Rent,Parking Lot,...,1,1,1,1,1,1,1,1,1,1
2,23150,33,F,0,82,79027,1,0,Rent,Local,...,1,1,1,1,1,1,1,1,1,1
3,19040,43,F,0,40,98544,1,0,Own,Parking Lot,...,1,1,1,1,1,1,1,1,1,1
4,10803,43,F,1,84,62619,1,1,Own,Parking Lot,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,29803,43,F,1,65,97505,1,0,Own,Local,...,1,1,1,1,1,1,1,1,1,1
17996,5391,33,F,0,65,81642,1,1,Own,Local,...,1,1,1,1,1,1,1,2,1,1
17997,861,39,M,0,84,123385,0,1,Own,Local,...,1,1,1,1,1,1,1,1,1,1
17998,15796,49,F,0,89,91848,1,0,Rent,Local,...,1,1,1,1,1,1,1,1,1,1


In [120]:
newname = 'subset_info_5'
new_subset_name = f'{newname}.csv'

df_with_features['fraud'] = target
fraud_diffs = compute_fraud_rate_differences(df_with_features, target_col='fraud', variance_threshold=0.05)
temp_1 = fraud_diffs[np.abs(fraud_diffs['difference']) > 0.03]

temp_1['info'] = (temp_1['variance']/0.25)*np.abs(temp_1['difference'])
temp_1 = temp_1.sort_values(by='info', ascending=False)
temp_1.to_csv(f'../../Create_Tune_Models/logs/{new_subset_name}', index = False)

Kept 14783 higher-variance features.


In [146]:
presence_info_df = temp_1

difference_min = 0.06
info_min = 0.043
presence_info_df = presence_info_df[
    (np.abs(presence_info_df['difference']) > difference_min) &
    (presence_info_df['info'] > info_min)
]

In [161]:
presence_info_df

,feature,fraud_rate_at_0,fraud_rate_at_1,difference,variance,info
1231,gender__marital_status__high_education_ind__li...,0.198295,0.133201,-0.065094,0.236647,0.061617
2861,marital_status__high_education_ind__living_sta...,0.197008,0.133040,-0.063968,0.238706,0.061078
1324,gender__marital_status__high_education_ind__ve...,0.197783,0.133357,-0.064426,0.237004,0.061077
10748,gender__marital_status__high_education_ind__li...,0.196589,0.134923,-0.061666,0.235088,0.057988
101,gender__marital_status__high_education_ind__ve...,0.215664,0.139610,-0.076054,0.184844,0.056232
1319,gender__marital_status__high_education_ind__ve...,0.199323,0.138156,-0.061166,0.220447,0.053936
5134,gender__marital_status__high_education_ind__li...,0.210109,0.140050,-0.070060,0.192117,0.053839
2933,marital_status__high_education_ind__accident_s...,0.199387,0.138312,-0.061075,0.219736,0.053681
5219,gender__marital_status__high_education_ind__li...,0.199720,0.138902,-0.060818,0.216767,0.052733
1262,gender__marital_status__high_education_ind__ac...,0.200035,0.139123,-0.060913,0.215250,0.052446


In [ ]:
low_card_col = filter_low_cardinality_columns(train_df, threshold=120)

custom_cols = ['claim_date.weekofyear', 'claim_date.quarter', 'log_pop_bin', 'vehicle_price_categories', 'zero_payout']
low_card_col = [col for col in low_card_col if col not in custom_cols]


# Step 2: Generate all 2- and 3-column combinations
combos_2345 = generate_column_combinations(low_card_col, [2, 3, 4, 5])

# Step 3: Add binary presence features
df_with_features_2345 = add_presence_features(train_df, combos_2345)

print("New dataset shape:", df_with_features_2345.shape)

In [ ]:
df_with_features_2345['fraud'] = target
fraud_diffs_2345 = compute_fraud_rate_differences(df_with_features_2345, target_col='fraud', variance_threshold=0.1)
temp_2 = fraud_diffs_2345[np.abs(fraud_diffs_2345['difference']) > 0.03]

temp_2['info'] = (temp_2['variance']/0.25)*np.abs(temp_2['difference'])
temp_2 = temp_2.sort_values(by='info', ascending=False)
temp_2.to_csv('../../Create_Tune_Models/logs/subset_info_2.csv', index = False)